In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

In [2]:
import kagglehub
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
# Download
path = kagglehub.dataset_download("aslanahmedov/walmart-sales-forecast")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\gleni\.cache\kagglehub\datasets\aslanahmedov\walmart-sales-forecast\versions\1


In [3]:
df_train = pd.read_csv(path + "/train.csv")
df_test = pd.read_csv(path + "/test.csv")
df_stores = pd.read_csv(path + "/stores.csv")
df_features = pd.read_csv(path + "/features.csv")

In [ ]:
df_test.index = pd.to_datetime(df_test['Date'])
df_test.sort_index(inplace=True)
df_test.drop(columns=['Date'], inplace=True)
df_test.head()

In [4]:
df = df_train.merge(df_features, on=["Store", "Date"], how="inner").merge(df_stores, on=["Store"], how="inner")

In [5]:
df_test = df_test.merge(df_features, on=["Store", "Date"], how="inner").merge(df_stores, on=["Store"], how="inner")

In [7]:
df_test.shape

(115064, 16)

In [8]:
df.shape

(421570, 17)

In [9]:
df['Date'] = pd.to_datetime(df['Date'])
df_test['Date'] = pd.to_datetime(df_test['Date'])

In [10]:
df['Week'] = df['Date'].dt.isocalendar().week
df_test['Week'] = df_test['Date'].dt.isocalendar().week

In [11]:
df['Day_of_week'] =df['Date'].dt.dayofweek
df_test['Day_of_week'] =df_test['Date'].dt.dayofweek

df['Month'] =df['Date'].dt.month 
df_test['Month'] =df_test['Date'].dt.month 

df['Year'] =df['Date'].dt.year
df_test['Year'] =df_test['Date'].dt.year

df['Day'] =df['Date'].dt.day 
df_test['Day'] =df_test['Date'].dt.day

In [15]:
df.tail()

,Store,Dept,Date,Weekly_Sales,IsHoliday_x,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,...,CPI,Unemployment,IsHoliday_y,Type,Size,Week,Day_of_week,Month,Year,Day
421565,45,98,2012-09-28,508.37,False,64.88,3.997,4556.61,20.64,1.50,...,192.013558,8.684,False,B,118221,39,4,9,2012,28
421566,45,98,2012-10-05,628.10,False,64.89,3.985,5046.74,NaN,18.82,...,192.170412,8.667,False,B,118221,40,4,10,2012,5
421567,45,98,2012-10-12,1061.02,False,54.47,4.000,1956.28,NaN,7.89,...,192.327265,8.667,False,B,118221,41,4,10,2012,12
421568,45,98,2012-10-19,760.01,False,56.47,3.969,2004.02,NaN,3.18,...,192.330854,8.667,False,B,118221,42,4,10,2012,19
421569,45,98,2012-10-26,1076.80,False,58.85,3.882,4018.91,58.08,100.00,...,192.308899,8.667,False,B,118221,43,4,10,2012,26


In [16]:
df_test.head()

,Store,Dept,Date,IsHoliday_x,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,...,CPI,Unemployment,IsHoliday_y,Type,Size,Week,Day_of_week,Month,Year,Day
0,1,1,2012-11-02,False,55.32,3.386,6766.44,5147.70,50.82,3639.90,...,223.462779,6.573,False,A,151315,44,4,11,2012,2
1,1,1,2012-11-09,False,61.24,3.314,11421.32,3370.89,40.28,4646.79,...,223.481307,6.573,False,A,151315,45,4,11,2012,9
2,1,1,2012-11-16,False,52.92,3.252,9696.28,292.10,103.78,1133.15,...,223.512911,6.573,False,A,151315,46,4,11,2012,16
3,1,1,2012-11-23,True,56.23,3.211,883.59,4.17,74910.32,209.91,...,223.561947,6.573,True,A,151315,47,4,11,2012,23
4,1,1,2012-11-30,False,52.34,3.207,2460.03,NaN,3838.35,150.57,...,223.610984,6.573,False,A,151315,48,4,11,2012,30


In [17]:
train = df.drop(columns=['IsHoliday_x'])
test = df_test.drop(columns=['IsHoliday_x'])

In [19]:
train['IsHoliday_y'] = train['IsHoliday_y'].astype(int)
test['IsHoliday_y'] = test['IsHoliday_y'].astype(int)

In [23]:
train['Type'] = train['Type'].map({'A': 1, 'B': 2, 'C': 3})
test['Type'] = test['Type'].map({'A': 1, 'B': 2, 'C': 3})

In [29]:
train.fillna(0, inplace=True)
#test.isnull().sum()

In [30]:
train.isnull().sum()

Store           0
Dept            0
Date            0
Weekly_Sales    0
Temperature     0
Fuel_Price      0
MarkDown1       0
MarkDown2       0
MarkDown3       0
MarkDown4       0
MarkDown5       0
CPI             0
Unemployment    0
IsHoliday_y     0
Type            0
Size            0
Week            0
Day_of_week     0
Month           0
Year            0
Day             0
dtype: int64

In [31]:
train.head()

,Store,Dept,Date,Weekly_Sales,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,...,CPI,Unemployment,IsHoliday_y,Type,Size,Week,Day_of_week,Month,Year,Day
0,1,1,2010-02-05,24924.50,42.31,2.572,0.0,0.0,0.0,0.0,...,211.096358,8.106,0,1,151315,5,4,2,2010,5
1,1,1,2010-02-12,46039.49,38.51,2.548,0.0,0.0,0.0,0.0,...,211.242170,8.106,1,1,151315,6,4,2,2010,12
2,1,1,2010-02-19,41595.55,39.93,2.514,0.0,0.0,0.0,0.0,...,211.289143,8.106,0,1,151315,7,4,2,2010,19
3,1,1,2010-02-26,19403.54,46.63,2.561,0.0,0.0,0.0,0.0,...,211.319643,8.106,0,1,151315,8,4,2,2010,26
4,1,1,2010-03-05,21827.90,46.50,2.625,0.0,0.0,0.0,0.0,...,211.350143,8.106,0,1,151315,9,4,3,2010,5


In [ ]:
train_y = train['Weekly_Sales']
train_x = train.drop(columns=['Weekly_Sales', 'Date'])

In [33]:
train_x.head()

,Store,Dept,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday_y,Type,Size,Week,Day_of_week,Month,Year,Day
0,1,1,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,0,1,151315,5,4,2,2010,5
1,1,1,38.51,2.548,0.0,0.0,0.0,0.0,0.0,211.242170,8.106,1,1,151315,6,4,2,2010,12
2,1,1,39.93,2.514,0.0,0.0,0.0,0.0,0.0,211.289143,8.106,0,1,151315,7,4,2,2010,19
3,1,1,46.63,2.561,0.0,0.0,0.0,0.0,0.0,211.319643,8.106,0,1,151315,8,4,2,2010,26
4,1,1,46.50,2.625,0.0,0.0,0.0,0.0,0.0,211.350143,8.106,0,1,151315,9,4,3,2010,5


In [34]:
test_x = test.drop(columns=['Date'])
test_x.head()

,Store,Dept,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday_y,Type,Size,Week,Day_of_week,Month,Year,Day
0,1,1,55.32,3.386,6766.44,5147.70,50.82,3639.90,2737.42,223.462779,6.573,0,1,151315,44,4,11,2012,2
1,1,1,61.24,3.314,11421.32,3370.89,40.28,4646.79,6154.16,223.481307,6.573,0,1,151315,45,4,11,2012,9
2,1,1,52.92,3.252,9696.28,292.10,103.78,1133.15,6612.69,223.512911,6.573,0,1,151315,46,4,11,2012,16
3,1,1,56.23,3.211,883.59,4.17,74910.32,209.91,303.32,223.561947,6.573,1,1,151315,47,4,11,2012,23
4,1,1,52.34,3.207,2460.03,NaN,3838.35,150.57,6966.34,223.610984,6.573,0,1,151315,48,4,11,2012,30


In [35]:
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_squared_error, r2_score


rf = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model
rf.fit(train_x, train_y)

# Make predictions
y_pred = rf.predict(test_x)


In [36]:
mse = mean_squared_error(train_y, rf.predict(train_x))
r2 = r2_score(train_y, rf.predict(train_x))

In [39]:
print(f"Mean Squared Error: {mse}")
print(f"R^2: {r2}")

Mean Squared Error: 1430085.0045633963
R^2: 0.9972274247015811


In [40]:
submission = pd.DataFrame({
        "Id": test.Store.astype(str)+'_'+test.Dept.astype(str)+'_'+test.Date.astype(str),
        "Weekly_Sales": y_pred
    })

submission.to_csv('submission predicted_RF_Final_3.csv', index=False)

In [43]:
submission.head()

,Id,Weekly_Sales
0,1_1_2012-11-02,33267.4243
1,1_1_2012-11-09,20834.0892
2,1_1_2012-11-16,19059.4367
3,1_1_2012-11-23,20578.5056
4,1_1_2012-11-30,26642.3270


In [44]:
submission.to_csv('submission.csv', index=False)

In [46]:
importance = rf.feature_importances_

In [48]:
importance = pd.DataFrame(importance, index=train_x.columns, columns=["Importance"])
importance.sort_values(by='Importance', ascending=False)

,Importance
Dept,0.629000
Size,0.189535
Store,0.057569
Week,0.036709
CPI,0.025382
Type,0.014176
Unemployment,0.010767
Temperature,0.008940
Day,0.008226
Month,0.005840


In [49]:
features = ['Store', 'Dept', 'IsHoliday_y', 'Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'Type', 'Size', 'Week', 'Month']	

In [50]:
train_x = train_x[features]
test_x = test_x[features]

In [51]:
rf2 = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model
rf2.fit(train_x, train_y)

# Make predictions
y_pred_2 = rf2.predict(test_x)

mse = mean_squared_error(train_y, rf2.predict(train_x))
r2 = r2_score(train_y, rf2.predict(train_x))

In [52]:
print(f"Mean Squared Error: {mse}")
print(f"R^2: {r2}")

Mean Squared Error: 1579393.516927516
R^2: 0.9969379530324821


In [53]:
feature_importance = rf2.feature_importances_

feature_importance = pd.DataFrame(feature_importance, index=train_x.columns, columns=["Importance"])
feature_importance.sort_values(by='Importance', ascending=False)

,Importance
Dept,0.628997
Size,0.190399
Store,0.057631
Week,0.042891
CPI,0.027457
Type,0.014250
Unemployment,0.011316
Temperature,0.011110
Month,0.008003
Fuel_Price,0.006161


In [ ]:
df['Type'] = le.fit_transform(df['Type'])


,Store,Dept,Date,Weekly_Sales,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,...,CPI,Unemployment,IsHoliday_y,Type,Size,Week,Day_of_week,Month,Year,Day
0,1,1,2010-02-05,24924.50,42.31,2.572,0.00,0.00,0.00,0.00,...,211.096358,8.106,0,1,151315,5,4,2,2010,5
1,1,1,2010-02-12,46039.49,38.51,2.548,0.00,0.00,0.00,0.00,...,211.242170,8.106,1,1,151315,6,4,2,2010,12
2,1,1,2010-02-19,41595.55,39.93,2.514,0.00,0.00,0.00,0.00,...,211.289143,8.106,0,1,151315,7,4,2,2010,19
3,1,1,2010-02-26,19403.54,46.63,2.561,0.00,0.00,0.00,0.00,...,211.319643,8.106,0,1,151315,8,4,2,2010,26
4,1,1,2010-03-05,21827.90,46.50,2.625,0.00,0.00,0.00,0.00,...,211.350143,8.106,0,1,151315,9,4,3,2010,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421565,45,98,2012-09-28,508.37,64.88,3.997,4556.61,20.64,1.50,1601.01,...,192.013558,8.684,0,2,118221,39,4,9,2012,28
421566,45,98,2012-10-05,628.10,64.89,3.985,5046.74,0.00,18.82,2253.43,...,192.170412,8.667,0,2,118221,40,4,10,2012,5
421567,45,98,2012-10-12,1061.02,54.47,4.000,1956.28,0.00,7.89,599.32,...,192.327265,8.667,0,2,118221,41,4,10,2012,12
421568,45,98,2012-10-19,760.01,56.47,3.969,2004.02,0.00,3.18,437.73,...,192.330854,8.667,0,2,118221,42,4,10,2012,19
